In [ ]:
import pandas as pd

path = "C:\\Users\\dmehri\\Documents\\DATA\\DOHMH\\"

df = pd.read_csv(path + "DOHMH_New_York_City_Restaurant_Inspection_Results 2013-2019.csv", low_memory=False)


Clean Data

In [ ]:
df["Community Board"] = df["Community Board"].astype(str)
df["Community Board"] = df['Community Board'].str.split(".").str[0]
df["ACTION"] = df["ACTION"].astype(str)
df["CUISINE DESCRIPTION"] = df["CUISINE DESCRIPTION"].astype(str)

Count Number of Restaurants by Cuisine

In [ ]:
pd.set_option('chained_assignment', None)


dfcount = df[["CUISINE DESCRIPTION"]]
dfcount["Count"] = 1
dfcount = dfcount.groupby(['CUISINE DESCRIPTION']).sum()
dfcount = dfcount.add_suffix('').reset_index()
dfcount = dfcount.sort_values(by = 'Count', ascending=False).reset_index(drop=True)

dfcount

Subset for Closures

In [ ]:
print ("len of df before drop", len(df))
df2 = df[df["ACTION"] == "Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed."]
print ("len of df after drop", len(df2))



In [ ]:
df3 = df2[["CUISINE DESCRIPTION"]]
df3["Count Closed"] = 1
df3 = df3.groupby(['CUISINE DESCRIPTION']).sum()
df3 = df3.add_suffix('').reset_index()
df3 = df3.sort_values(by = 'Count Closed', ascending=False).reset_index(drop=True)



Merge with Count

In [ ]:
df3 = pd.merge(df3, dfcount, how='left', on=['CUISINE DESCRIPTION'])
df3["Count"] = df3["Count"].astype(float)


In [ ]:
df3["Percent Closed"] = (df3["Count Closed"]/df3["Count"] )*100
df3 = df3.sort_values(by = 'Percent Closed', ascending=False).reset_index(drop=True)

In [ ]:
df3.to_csv(path + "Restaurants Closed by Cuisine.csv", index=False)

Community Board

In [ ]:
dfcb = df2[["Community Board"]]
dfcb["Count Closed"] = 1
dfcb = dfcb.groupby(['Community Board']).sum()
dfcb = dfcb.add_suffix('').reset_index()
dfcb = dfcb.sort_values(by = 'Count Closed', ascending=False).reset_index(drop=True)


In [ ]:
dfcb2 = df[["Community Board"]]
dfcb2["Number of Inspections"] = 1
dfcb2 = dfcb2.groupby(['Community Board']).sum()
dfcb2 = dfcb2.add_suffix('').reset_index()
dfcb2 = dfcb2.sort_values(by = 'Number of Inspections', ascending=False).reset_index(drop=True)


In [ ]:
dfcb = pd.merge(dfcb, dfcb2, how='left', on=['Community Board'])


In [ ]:
dfcb["Number of Inspections"] = dfcb["Number of Inspections"].astype(float)
dfcb["Percent Closed"] = (dfcb["Count Closed"]/dfcb["Number of Inspections"])*100
dfcb = dfcb.sort_values(by = 'Percent Closed', ascending=False).reset_index(drop=True)


Top Cuisines Closed by Community Board

In [ ]:
cb = sorted(list(set( df2["Community Board"].tolist() )))


qn = pd.DataFrame(columns=('Community Board', 'Top 5 Closed'))


for i in range(0, len(cb)):
    df3  = df2[df2["Community Board"] == cb[i]]
    df3 = df3[["CUISINE DESCRIPTION"]]
    df3["Count"] = 1
    df3 = df3.groupby(['CUISINE DESCRIPTION']).sum()
    df3 = df3.add_suffix('').reset_index()
    df3 = df3.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
    
    df3 = df3[:5]
    
    top5 = df3["CUISINE DESCRIPTION"].tolist()
    #print (cb[i], top5)
    
    qn.loc[i] = [cb[i],top5]



    
qn["Top 5 Closed"] = qn["Top 5 Closed"].astype(str)
qn["Top 5 Closed"] = qn["Top 5 Closed"].str.replace('[', '')
qn["Top 5 Closed"] = qn["Top 5 Closed"].str.replace(']', '')
qn["Top 5 Closed"] = qn["Top 5 Closed"].str.replace("'", '')


Count by Borough

In [ ]:
set(df2["BORO"].tolist() )


dfb = df2[["BORO"]]
dfb["Count"] = 1
dfb = dfb.groupby(['BORO']).sum()
dfb = dfb.add_suffix('').reset_index()

dfb.to_csv(path + "Restaurant Closure by Borouogh.csv", index=False)

In [ ]:
dfcb = pd.merge(dfcb, qn, how='left', on=['Community Board'])


In [ ]:
dfcb.to_csv(path + "Restaurant Closure CB.csv", index=False)